In [1]:
from google.colab import drive
drive.mount("/F")

Mounted at /F


In [2]:
import pandas as pd
base = pd.read_csv('/F/My Drive/Maratona/Autoencoders/credit_data.csv')

In [3]:
base = base.drop('i#clientid', axis = 1)
base = base.dropna()
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
X = base.drop('c#default', axis = 1)
y = base['c#default']
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [4]:
neuronios_entrada = 3
neuronios_oculta = 2
neuronios_saida = neuronios_entrada

In [5]:
import tensorflow as tf

In [6]:
camada_oculta = tf.keras.layers.Dense(units = neuronios_oculta, activation = None)
camada_saida = tf.keras.layers.Dense(units = neuronios_saida, activation = None)

In [7]:
X = tf.convert_to_tensor(X)

In [8]:
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [9]:
for epoca in range(1000):
    with tf.GradientTape() as tape:
        X2d_encode = camada_oculta(X)
        X3d_decode = camada_saida(X2d_encode)
        erro = tf.keras.losses.MSE(X, X3d_decode)

    # A lista de variáveis treináveis só é criada depois que o modelo já processou
    # um batch, pois a partir dos dados de entrada ele deduz os formatos das matrizes
    # de peso e bias, por isso só estou criando a variável `var_list` aqui dentro
    # do loop de treinamento
    var_list = [*camada_oculta.trainable_variables, *camada_saida.trainable_variables]
    
    gradientes = tape.gradient(erro, var_list)
    
    otimizador.apply_gradients(zip(gradientes, var_list))

    if epoca % 100 == 0:
        print('erro: ' + str(tf.reduce_mean(erro).numpy()))

erro: 2.5030024
erro: 0.4666015
erro: 0.26830682
erro: 0.20928632
erro: 0.19411857
erro: 0.18918833
erro: 0.1871451
erro: 0.18626076
erro: 0.18589015
erro: 0.18574749


In [10]:
var_list

[<tf.Variable 'dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.11907663,  0.8257465 ],
        [-0.39788523, -0.17948847],
        [-0.15972076,  0.8118523 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0.00231556, 0.00344029], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.13887595, -2.322375  , -0.37194902],
        [ 0.5854757 , -0.3955427 ,  0.548841  ]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(3,) dtype=float32, numpy=array([-0.00168987,  0.00672397, -0.00102551], dtype=float32)>]

In [11]:
X2d_encode.shape

TensorShape([1997, 2])

In [12]:
X3d_decode.shape

TensorShape([1997, 3])

In [13]:
X2 = scaler_x.inverse_transform(X.numpy())
X2

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [14]:
X3d_decode2 = scaler_x.inverse_transform(X3d_decode.numpy())
X3d_decode2

array([[6.3353090e+04, 5.8749920e+01, 8.6992051e+03],
       [4.4448336e+04, 4.9073956e+01, 4.4413691e+03],
       [5.8519047e+04, 6.3222958e+01, 7.7660767e+03],
       ...,
       [4.8013402e+04, 2.8369797e+01, 4.7396538e+03],
       [3.6727352e+04, 6.3302330e+01, 3.1096501e+03],
       [6.3441477e+04, 5.5581085e+01, 8.6471152e+03]], dtype=float32)

In [15]:
from sklearn.metrics import mean_absolute_error
mae_income = mean_absolute_error(X2[:, 0], X3d_decode2[:, 0])
mae_income

6259.229227955363

In [16]:
mae_age = mean_absolute_error(X2[:, 1], X3d_decode2[:, 1])
mae_age

0.596775528478623

In [17]:
mae_loan = mean_absolute_error(X2[:, 2], X3d_decode2[:, 2])
mae_loan

1324.5646987930797

In [18]:
X_encode = pd.DataFrame({'atributo1': X2d_encode[:, 0], 'atributo2': X2d_encode[:, 1], 'classe': y})

In [19]:
X_encode.head()

,atributo1,atributo2,classe
0,-0.894705,1.939750,0
1,-0.231582,-0.157624,0
2,-0.936240,1.353393,0
3,-0.207264,0.228893,0
4,0.244942,2.695470,1


In [20]:
colunas = [tf.feature_column.numeric_column(key = column) for column in X_encode.columns]
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_encode, y, test_size = 0.3, random_state = 42)

In [21]:
def make_input_fn(X, y = None, num_epochs = None, shuffle = True, batch_size = 32):
    def input_function():
        # As variáveis previsoras precisam ser passadas na função seguinte como um dicionário com os mesmos nomes definidos na chamada de `tf.feature_column.numeric_column`
        ds = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [22]:
funcao_treinamento = make_input_fn(X_treinamento, y_treinamento, batch_size = 8)
config = tf.estimator.RunConfig(tf_random_seed = 42)
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4], config = config)
classificador.train(input_fn = funcao_treinamento, steps = 1000)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpz3bu16km', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and grap

In [23]:
funcao_teste = make_input_fn(X_teste, y_teste, num_epochs = 1000, shuffle = False, batch_size = 8)
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-27T19:20:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpz3bu16km/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Inference Time : 1.43695s
INFO:tensorflow:Finished evaluation at 2021-01-27-19:20:50
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.895125, accuracy_baseline = 0.848875, auc = 0.9008923, auc_precision_recall = 0.7170734, average_loss = 0.57542557, global_step

In [24]:
metricas_teste

{'accuracy': 0.895125,
 'accuracy_baseline': 0.848875,
 'auc': 0.9008923,
 'auc_precision_recall': 0.7170734,
 'average_loss': 0.57542557,
 'global_step': 1000,
 'label/mean': 0.151125,
 'loss': 0.57542557,
 'precision': 0.8886555,
 'prediction/mean': 0.43165872,
 'recall': 0.34987593}